In [76]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
import csv
from sklearn.metrics import confusion_matrix

from sklearn.naive_bayes import MultinomialNB

In [77]:
dataset=[]
real_labels=[]
count=0

with open('sentiment-analysis.csv', newline='') as csvfile:
    
    reader = csv.reader(csvfile)
    
    for row in reader:
        if(count!=0):
          result=(str(row[0]).split(","))
          if(len(result)>1): 
            dataset.append(result[0])
            sentiment=result[1]
            if(sentiment[1] == 'P'):
                real_labels.append(1)
            else:
                real_labels.append(0)
        count+=1

In [78]:
# vetorização dos textos
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(dataset).toarray()

In [79]:
# 30% do conjunto de dados é utilizado para os testes e 70% é utilizado para o treinamento
x_train, x_test, y_train, y_test = train_test_split(X, real_labels, test_size=0.3, random_state=42)

In [80]:

mnb = MultinomialNB()

mnb.fit(x_train, y_train)
y_pred_mnb = mnb.predict(x_test)

# avaliação do modelo
print(classification_report(y_test, y_pred_mnb))
conf_matrix = confusion_matrix(y_test, y_pred_mnb)

# Criar um DataFrame com 16 linhas e duas colunas
data = {'Verdadeiro': ['Positivo' if i==1 else 'Negativo' for i in y_test], 'Previsto': ['Positivo' if i==1 else 'Negativo' for i in y_pred_mnb]}
df = pd.DataFrame(data)
display(df)
        

              precision    recall  f1-score   support

           0       0.73      1.00      0.85        11
           1       1.00      0.78      0.88        18

    accuracy                           0.86        29
   macro avg       0.87      0.89      0.86        29
weighted avg       0.90      0.86      0.86        29



,Verdadeiro,Previsto
0,Negativo,Negativo
1,Positivo,Positivo
2,Positivo,Positivo
3,Negativo,Negativo
4,Positivo,Positivo
5,Positivo,Negativo
6,Positivo,Negativo
7,Positivo,Positivo
8,Positivo,Negativo
9,Positivo,Positivo
